In [1]:
import cv2                 
import numpy as np         
import os                  
from random import shuffle 
from tqdm import tqdm      
##TRAIN_DIR = 'E:/P_2020/PlantDisease-master/leaf-disease/Datasets/D'
##TEST_DIR = 'E:/P_2020/PlantDisease-master/leaf-disease/Datasets/D/test'

TRAIN_DIR = 'C:/Users/acer/Desktop/PlantDisease-master(working)/test/test'
TEST_DIR = 'C:/Users/acer/Desktop/PlantDisease-master(working)/train/train'

IMG_SIZE = 50
LR = 1e-3
MODEL_NAME = 'healthyvsunhealthy-{}-{}.model'.format(LR, '2conv-basic')

def label_img(img):
    word_label = img[0]
    print(word_label)
  
    if word_label == 'h':
        print('healthy')
        return [1,0,0,0]
    
    elif word_label == 'b':
        print('Bacterial')
        return [0,1,0,0]
    elif word_label == 'v':
        print('virus')
        return [0,0,1,0]
##    elif word_label == 'l':
##        print('lllll')
##        return [0,0,0,1]

    

def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        print('##############')
        print(label)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

train_data = create_train_data()
# If you have already created the dataset:
#train_data = np.load('train_data.npy')


import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf
tf.reset_default_graph()

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')

convnet = conv_2d(convnet, 32, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 128, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 32, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = conv_2d(convnet, 64, 3, activation='relu')
convnet = max_pool_2d(convnet, 3)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 4, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
test_y = [i[1] for i in test]

model.fit({'input': X}, {'targets': Y}, n_epoch=8, validation_set=({'input': test_x}, {'targets': test_y}),
    snapshot_step=40, show_metric=True, run_id=MODEL_NAME)

model.save(MODEL_NAME)











        


  0%|                                                                                           | 0/80 [00:00<?, ?it/s]

b
Bacterial
##############
[0, 1, 0, 0]


  1%|█                                                                                  | 1/80 [00:00<00:15,  5.10it/s]

b
Bacterial
##############
[0, 1, 0, 0]


  2%|██                                                                                 | 2/80 [00:00<00:14,  5.42it/s]

...

h
healthy
##############
[1, 0, 0, 0]


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:11<00:00,  7.25it/s]


curses is not supported on this machine (please install/reinstall curses for an optimal experience)







Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\acer\Desktop\PlantDisease-master(working)\healthyvsunhealthy-0.001-2conv-basic.model
model loaded!
--------

Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\acer\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\acer\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\acer\Anaconda3\lib\site-packages\tflearn\data_flow.py", line 201, in fill_batch_ids_queue
    ids = self.next_batch_ids()
  File "C:\Users\acer\Anaconda3\lib\site-packages\tflearn\data_flow.py", line 215, in next_batch_ids
    batch_start, batch_end = self.batches[self.batch_index]
IndexError: list index out of range



INFO:tensorflow:C:\Users\acer\Desktop\PlantDisease-master(working)\healthyvsunhealthy-0.001-2conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.
